In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import neurd

# Parameter Tutorial (housed inside the vdi)

In [ ]:
from neurd import parameter_utils as paru

In [ ]:
filepath = "../../neurd/parameter_configs/parameters_config_default.py"
filepath

In [ ]:
obj = paru.parameters_from_filepath(
    filepath = filepath
)

obj

In [ ]:
print(obj)

In [ ]:
from neurd import soma_extraction_utils as sm
paru.parameter_list_from_module(
    sm
)

In [ ]:
print(obj["apical_utils"])

In [ ]:
paru.parameter_dict_from_module_and_obj(
    apu,
    vdi
)

In [ ]:
paru.set_parameters_for_directory_modules_from_obj(
    vdi,
    directory = "../../neurd/",
    verbose_loop=True,
    verbose_param=True,
    error_on_no_attr=True
)

In [ ]:
obj.module_attr_map(
    "soma_extraction_utils"
)

In [ ]:
"""
Behind the scenes workflow:

1) Module request parameters
2) parameter_obj generates list of parameters settings (dict)
3) setattr of module with parameter setting dict

"""

In [ ]:
paru.parameter_list_from_module(
    sm
)

# Volume Data Interface

In [ ]:
from neurd.vdi_microns import volume_data_interface as vdi
vdi.set_parameters_for_directory_modules()

# Output Storage

In [ ]:
from python_tools import pipeline

In [ ]:
p_obj = pipeline.PipelineProducts(
    {"decimation":{
        'decimation_ratio':0.25,
        'mesh_original':None
        },
     'soma_identification':{
         'number_of_somas':2,
     }
    }
)

p_obj

In [ ]:
print(p_obj)

In [ ]:
print(p_obj['decimation'])#['decimation_ratio']

# Using Neurd

In [ ]:
import neurd

In [ ]:
from neurd.vdi_microns import volume_data_interface as vdi
vdi.set_parameters_for_directory_modules()

# Data Prep: Loading mesh

In [ ]:
segment_id = 123

In [ ]:
mesh = vdi.fetch_segment_id_mesh(
    segment_id,
    meshes_folder="../autoproof_neuron_pipeline/",
    ext="off",
    plot = True,
)
mesh

In [ ]:
from python_tools import pipeline

products = pipeline.PipelineProducts()
products

# Step 1: Decimation

In [ ]:
from mesh_tools import trimesh_utils as tu

In [ ]:
decimation_parameters = dict(
    decimation_ratio =0.25,
)

In [ ]:
mesh_decimated = tu.decimate(
    mesh,
    **decimation_parameters
)

In [ ]:
mesh,mesh_decimated

In [ ]:
products.set_stage_attrs(
    stage = "decimation",
    attr_dict = dict(
        decimation_parameters = decimation_parameters,
        original_mesh = mesh,
    ),
    
)

print(products)

# Step 2: Soma Identification

In [ ]:
from neurd import soma_extraction_utils as sm

In [ ]:
soma_extraction_parameters = dict()

In [ ]:
#optional

from neurd import parameter_utils as paru
paru.category_param_from_module(
        module = sm,
        category = "glia",
)

In [ ]:
verbose = True

soma_products = sm.soma_indentification(
    mesh,
    verbose=verbose,
    **soma_extraction_parameters
)

In [ ]:
print(soma_products)

In [ ]:
from python_tools import ipyvolume_utils as ipvu
ipvu.plot_objects(
    meshes = soma_products.soma_meshes
)

In [ ]:
sm.plot_soma_products(
    mesh,
    soma_products = soma_products,
    verbose = True
)

In [ ]:
soma_products.soma_extraction_parameters = soma_extraction_parameters
soma_products

In [ ]:
products.set_stage_attrs(
    stage = "soma_identification",
    attr_dict = soma_products,
)

print(products)

# Decomposition

In [ ]:
decomposition_parameters = {}

In [ ]:
from neurd import neuron

neuron_obj = neuron.Neuron(
    mesh = mesh,
    pipeline_products = products,
    suppress_preprocessing_print=False,
    suppress_output=False,
    **decomposition_parameters
)

In [ ]:
from neurd import neuron_visualizations as nviz
nviz.visualize_neuron(
    neuron_obj,
    limb_branch_dict="all"
)

In [ ]:
neuron_obj.calculate_decomposition_products(
    store_in_obj = True,
)

In [ ]:
print(neuron_obj.pipeline_products)

# Saving off neuron

In [ ]:
from neurd import neuron_utils as nru

filepath = nru.save_compressed_neuron(
    neuron_obj,
    output_folder = "./",
    file_name = f"{segment_id}",
    return_file_path = True,
)